In [4]:
import numpy as np
from scipy.integrate import quad

In [3]:
# S0 = 100.0    # Initial stock price
# K = 100.0     # Strike price
# r = 0.05      # Risk-free rate
# T = 1.0       # Time to maturity
# kappa = 2.0   # Mean reversion rate
# theta = 0.05  # Long-term average volatility
# sigma = 0.3   # Volatility of volatility
# rho = -0.5    # Correlation coefficient
# v0 = 0.05     # Initial volatility5

S0 = 100.0
K = 100.0
r = 0.02
T = 1.0
kappa = 6.21
theta = 0.04
sigma = 0.50
rho = -0.70
v0 = 0.04

In [5]:
# Define characteristic functions
def heston_characteristic_function(u, S0, K, r, T, kappa, theta, sigma, rho, v0):
   xi = kappa - rho * sigma * 1j * u
   d = np.sqrt((rho * sigma * 1j * u - xi)**2 - sigma**2 * (-u * 1j - u**2))
   g = (xi - rho * sigma * 1j * u - d) / (xi - rho * sigma * 1j * u + d)
   C = r * 1j * u * T + (kappa * theta) / sigma**2 * ((xi - rho * sigma * 1j * u - d) * T - 2 * np.log((1 - g * np.exp(-d * T)) / (1 - g)))
   D = (xi - rho * sigma * 1j * u - d) / sigma**2 * ((1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T)))
   return np.exp(C + D * v0 + 1j * u * np.log(S0))

# Define functions to compute call and put options prices
def heston_call_price(S0, K, r, T, kappa, theta, sigma, rho, v0):
   integrand = lambda u: np.real(np.exp(-1j * u * np.log(K)) / (1j * u) * heston_characteristic_function(u - 1j, S0, K, r, T, kappa, theta, sigma, rho, v0))
   integral, _ = quad(integrand, 0, np.inf)
   return np.exp(-r * T) * 0.5 * S0 - np.exp(-r * T) / np.pi * integral


def heston_put_price(S0, K, r, T, kappa, theta, sigma, rho, v0):
   integrand = lambda u: np.real(np.exp(-1j * u * np.log(K)) / (1j * u) * heston_characteristic_function(u - 1j, S0, K, r, T, kappa, theta, sigma, rho, v0))
   integral, _ = quad(integrand, 0, np.inf)
   return np.exp(-r * T) / np.pi * integral - S0 + K * np.exp(-r * T)

In [6]:
# Calculate call and put option prices
call_price = heston_call_price(S0, K, r, T, kappa, theta, sigma, rho, v0)
put_price = heston_put_price(S0, K, r, T, kappa, theta, sigma, rho, v0)


print("European Call Option Price:", np.round(call_price, 2))
print("European Put Option Price:", np.round(put_price, 2))

European Call Option Price: 31.53
European Put Option Price: 15.5


/var/folders/5r/6f18fbm17wl8t8q50sb5hrvc0000gn/T/ipykernel_52544/2917226655.py:13: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral, _ = quad(integrand, 0, np.inf)
/var/folders/5r/6f18fbm17wl8t8q50sb5hrvc0000gn/T/ipykernel_52544/2917226655.py:19: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral, _ = quad(integrand, 0, np.inf)


In [7]:
import numpy as np

class HestonParameters:
    def __init__(self, kappa, theta, sigma, rho, xi, S0=100.0, K=100.0, r = 0.02):
        self.kappa = kappa  # Mean reversion rate
        self.theta = theta  # Long-term average volatility
        self.sigma = sigma  # Volatility of volatility
        self.rho = rho      # Correlation coefficient
        self.S0 = S0       # Initial stock price
        self.K = K         # Strike price

def construct_generator_CIR(v, p: HestonParameters, reflect_left=True, reflect_right=True):
    """
    Construct Q generator matrix for CIR process
    
    Args:
        v: Vector of grid points (volatility values)
        p: Heston parameters (kappa, theta, sigma)
        reflect_left: Whether to use reflecting boundary at left
        reflect_right: Whether to use reflecting boundary at right
    
    Returns:
        Q: Generator matrix
    """
    m = len(v)
    assert m >= 3, "Need at least 3 grid points"
    assert all(np.diff(v) > 0), "Grid must be strictly increasing"

    Q = np.zeros((m, m))
    h = np.diff(v)  # h[i] = v[i+1] - v[i]
    
    # Define CIR drift and volatility functions
    def cir_drift(x, p):
        return p.kappa * (p.theta - x)
    
    def cir_sigma(x, p):
        return p.sigma * np.sqrt(x)
    
    def μ_plus(x):
        return max(cir_drift(x, p), 0.0)
    
    def μ_minus(x):
        return max(-cir_drift(x, p), 0.0)
    
    def σ_squared(x):
        return cir_sigma(x, p)**2
    
    # Interior points i = 1..m-2 (0-indexed)
    for i in range(1, m-1):
        hL, hR = h[i-1], h[i]
        mp, mm = μ_plus(v[i]), μ_minus(v[i])
        s2 = σ_squared(v[i])
        # MCAM / local-generator stencil
        qL = mm/hL + (s2 - (hL*mm + hR*mp)) / (hL*(hL + hR))
        qR = mp/hR + (s2 - (hL*mm + hR*mp)) / (hR*(hL + hR))
        # Guard small negatives that can occur on pathological grids
        qL = max(qL, 0.0)
        qR = max(qR, 0.0)
        Q[i, i-1] = qL
        Q[i, i+1] = qR
        Q[i, i] = -(qL + qR)
    
    # Left boundary i=0
    if reflect_left:
        hR = h[0]
        mp, s2 = μ_plus(v[0]), σ_squared(v[0])
        qR = mp/hR + s2/(hR**2)
        qR = max(qR, 0.0)
        Q[0, 1] = qR
        Q[0, 0] = -qR
    else:
        # absorbing at left
        Q[0, 0] = 0.0
    
    # Right boundary i=m-1
    if reflect_right:
        hL = h[-1]
        mm, s2 = μ_minus(v[-1]), σ_squared(v[-1])
        qL = mm/hL + s2/(hL**2)
        qL = max(qL, 0.0)
        Q[m-1, m-2] = qL
        Q[m-1, m-1] = -qL
    else:
        Q[m-1, m-1] = 0.0
    
    # Sanity: rows sum ~ 0
    assert np.max(np.abs(np.sum(Q, axis=1))) <= 1e-10, "Row sums not ~ 0; check grid/stencil"
    return Q


In [10]:
import matplotlib.pyplot as plt

def check_generator_negativity(v, p, h_minus, h_plus):
    # Define CIR drift and volatility functions
    def cir_drift(x, p):
        return p.kappa * (p.theta - x)
    
    def cir_sigma(x, p):
        return p.sigma * np.sqrt(x)
    
    def μ_plus(x):
        return max(cir_drift(x, p), 0.0)
    
    def μ_minus(x):
        return max(-cir_drift(x, p), 0.0)
    
    def σ_squared(x):
        return cir_sigma(x, p)**2
    
    # Calculate qL and qR without the max safeguard
    hL, hR = h_minus, h_plus
    mp, mm = μ_plus(v), μ_minus(v)
    s2 = σ_squared(v)
    
    qL = mm/hL + (s2 - (hL*mm + hR*mp)) / (hL*(hL + hR))
    qR = mp/hR + (s2 - (hL*mm + hR*mp)) / (hR*(hL + hR))
    
    return qL, qR

# Create arrays to store results
qL_values = np.zeros_like(v_values)
qR_values = np.zeros_like(v_values)

# Calculate qL and qR for each volatility value
for i, v in enumerate(v_values):
    qL_values[i], qR_values[i] = check_generator_negativity(v, PS_1, h_minus, h_plus)

# Plot the results
plt.figure(figsize=(12, 6))

# Plot qL and qR
plt.plot(v_values, qL_values, label='qL')
plt.plot(v_values, qR_values, label='qR')

# Highlight negative regions
plt.fill_between(v_values, qL_values, 0, where=(qL_values < 0), color='red', alpha=0.3, label='qL < 0')
plt.fill_between(v_values, qR_values, 0, where=(qR_values < 0), color='blue', alpha=0.3, label='qR < 0')

plt.axhline(y=0, color='k', linestyle='-', alpha=0.5)
plt.grid(True)
plt.xlabel('Volatility (v)')
plt.ylabel('Generator values')
plt.legend()
plt.title('Regions where volatility generator goes negative')

# Zoom in on interesting regions
plt.ylim(-10, 100)
plt.xlim(0, 0.5)  # Focus on relevant volatility range

plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/rishabhkumar/miniconda3/envs/bci_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/rishabhkumar/miniconda3/envs/bci_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/rishabhkumar/miniconda3/envs/bci_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/rishabhkumar/miniconda3/envs/bci_env/lib/python3.10/site-packages/trait

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import